In [2]:
from glob import glob
import numpy as np

In [7]:
def fix_annotation_file(file_path):
    def iou(box1, box2):
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2
        xi1 = max(x1 - w1 / 2, x2 - w2 / 2)
        yi1 = max(y1 - h1 / 2, y2 - h2 / 2)
        xi2 = min(x1 + w1 / 2, x2 + w2 / 2)
        yi2 = min(y1 + h1 / 2, y2 + h2 / 2)
        wi = max(0, xi2 - xi1)
        hi = max(0, yi2 - yi1)
        intersection = wi * hi
        area1 = w1 * h1
        area2 = w2 * h2
        union = area1 + area2 - intersection
        return intersection / union if union != 0 else 0
    with open(file_path, 'r') as file:
        lines = file.readlines()
    processed_boxes = {}
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id, cx, cy, w, h = parts
            cx, cy, w, h = map(float, [cx, cy, w, h])
            cx, cy, w, h = np.clip([cx, cy, w, h], 0, 1)
            cx, cy, w, h = [round(x, 4) for x in [cx, cy, w, h]]
            box = (cx, cy, w, h)
            if class_id not in processed_boxes:
                processed_boxes[class_id] = [box]
            else:
                add_box = True
                for existing_box in processed_boxes[class_id]:
                    if iou(box, existing_box) > 0.5:
                        add_box = False
                        break
                if add_box:
                    processed_boxes[class_id].append(box)
    with open(file_path, 'w') as file:
        for class_id, boxes in processed_boxes.items():
            for cx, cy, w, h in boxes:
                file.write(f"{class_id} {cx} {cy} {w} {h}\n")


In [10]:
labels = glob("data/*/*/*.txt")
len(labels)

5454

In [11]:
for label in labels: fix_annotation_file(label)